## Data Description

This dataset contains an anonymized set of features, feature_{0...129}, representing real stock market data. Each row in the dataset represents a trading opportunity, for which you will be predicting an action value: 1 to make the trade and 0 to pass on it. Each trade has an associated weight and resp, which together represents a return on the trade. The date column is an integer which represents the day of the trade, while ts_id represents a time ordering. In addition to anonymized feature values, you are provided with metadata about the features in features.csv.

In the training set, train.csv, you are provided a resp value, as well as several other resp_{1,2,3,4} values that represent returns over different time horizons. These variables are not included in the test set. Trades with weight = 0 were intentionally included in the dataset for completeness, although such trades will not contribute towards the scoring evaluation.


For extensive data analysis for the jane street market dataset go to following link [EDA-Quantitative-Researcher-prespective](https://www.kaggle.com/huzzefakhan/eda-quantitative-researcher-prespective) this notebook will go through the train and features csv's for an extensive exploratory data analysis, Also some data cleaning and preprocessing will be done along the way. In this note book I use Principal Components Analysis (PCA) to identify features to be used for supervised learning. This is first version of modeling i am also tring to add some finding from my EDA in this modeling exercise.


## About me

Working as Data Scientist in IT firm in Pakistan. I was Recently Enguaged with Radix Trading LLC which is a firm just like Jane Street which also work in High frequency algorithmic trading. Where i worked as Quantitative Researcher (Quant) to Capture Price movement in High frequency Algorithmic trading through Alphas. Designed many successful Alpha/strategies which is trade-able in real Stock market. For more details kindly visit my linkedin profile.

Please upvote if you find this notebook helpful! 😊 Thank you!.


In [ ]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
import optuna

In [ ]:
# Import dataset as train
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')#, nrows=2000000
train.info()

In [ ]:
# Drop rows with 'weight'=0 
# such trades will not contribute towards the scoring evaluation
train = train[train['weight']!=0]

# Create 'action' column (dependent variable)
train['action'] = train['resp'].apply(lambda x:x>0).astype(int)

In [ ]:
features = [col for col in list(train.columns) if 'feature' in col]

In [ ]:
X = train[features]
y = train['action']

# Next, we hold out part of the training data to form the hold-out validation set
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2)

In [ ]:
# First, we want to check if the target class is balanced or unbalanced in the training data
sns.set_palette("colorblind")
ax = sns.barplot(train_y.value_counts().index, train_y.value_counts()/len(train_y))
ax.set_title("Proportion of trades with action=0 and action=1")
ax.set_ylabel("Percentage")
ax.set_xlabel("Action")
sns.despine();
# Target class is fairly balanced with almost 50% of trades corresponding to each action

In [ ]:
# Finally, we investigate if there are missing values and we impute them
n_features = 60
nan_val = train.isna().sum()[train.isna().sum() > 0].sort_values(ascending=False)
print(nan_val)
fig, axs = plt.subplots(figsize=(10, 10))
sns.barplot(y = nan_val.index[0:n_features], 
            x = nan_val.values[0:n_features], 
            alpha = 0.8
           )
plt.title('Missing values of train dataset')
plt.xlabel('# of Missing values')
plt.show()
# There are quite a lot of missing values across the features


In [ ]:
train_median = train_x.median()
# Impute medians in both training set and the hold-out validation set
train_x = train_x.fillna(train_median)
valid_x = valid_x.fillna(train_median)

In [ ]:
# Before we perform PCA, we need to normalise the features so that they have zero mean and unit variance
scaler = StandardScaler()
scaler.fit(train_x)
train_x_norm = scaler.transform(train_x)

pca = PCA()
comp = pca.fit(train_x_norm)

# We plot a graph to show how the explained variation in the 129 features varies with the number of principal components
plt.plot(np.cumsum(comp.explained_variance_ratio_))
plt.grid()
plt.xlabel('Number of Principal Components')
plt.ylabel('Explained Variance')
sns.despine();

# The first 15 principal components explains about 80% of the variation
# The first 40 principal components explains about 95% of the variation

In [ ]:
# Using the first 40 principal components, we apply the PCA mapping
# From here on, we work with only 40 features instead of the full set of 129 features
pca = PCA(n_components=40).fit(train_x_norm)
train_x_transform = pca.transform(train_x_norm)

In [ ]:
valid_x_transform = pca.transform(scaler.transform(valid_x))

In [ ]:
# We create the XGboost-specific DMatrix data format from the numpy array. 
# This data structure is optimised for memory efficiency and training speed
dtrain = xgb.DMatrix(train_x_transform, label=train_y)
dvalid = xgb.DMatrix(valid_x_transform, label=valid_y)



In [ ]:
def objective(trial):
    
# params specifies the XGBoost hyperparameters to be tuned
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'max_depth': trial.suggest_int('max_depth', 10, 25),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'subsample': trial.suggest_uniform('subsample', 0.50, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.50, 1),
        'gamma': trial.suggest_int('gamma', 0, 10),
        'tree_method': 'gpu_hist',  
        'objective': 'binary:logistic'
    }
    
    bst = xgb.train(params, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
best_params = trial.params
best_params['tree_method'] = 'gpu_hist' 
best_params['objective'] = 'binary:logistic'

In [ ]:
# Fit the XGBoost classifier with optimal hyperparameters
clf = xgb.XGBClassifier(**best_params)

In [ ]:
clf.fit(train_x_transform, train_y)

In [ ]:
# Plot how the best accuracy evolves with number of trials
fig = optuna.visualization.plot_optimization_history(study)
fig.show();

In [ ]:
# We can also plot the relative importance of different hyperparameter settings
fig = optuna.visualization.plot_param_importances(study)
fig.show();

In [ ]:
# We impute the missing values with the medians
def fillna_npwhere(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    wt = test_df.iloc[0].weight
    if(wt == 0):
        sample_prediction_df.action = 0 
    else:
        sample_prediction_df.action = clf.predict(pca.transform(scaler.transform(fillna_npwhere(test_df[features].values,train_median[features].values))))
    env.predict(sample_prediction_df)

### Referrences

* https://www.kaggle.com/marketneutral/jane-street-eda-regime-tags-clusters
* https://www.kaggle.com/wongguoxuan/eda-pca-xgboost-classifier-for-beginners#3 
* https://www.kaggle.com/huzzefakhan/eda-quantitative-researcher-prespective
